# Forda-Fulkersona Algorithm
The Ford-Fulkerson algorithm is an algorithm that tackles the max-flow min-cut problem. That is, given a network with vertices and edges between those vertices that have certain weights, how much "flow" can the network process at a time? Flow can mean anything, but typically it means data through a computer network.

In [1]:
import numpy as np
from numpy import inf
from operator import itemgetter

In [2]:
# Set the file name
filename = "graf3.txt"

In [3]:
# Open file and create 
with open(filename, 'r') as myfile:
    content = myfile.readlines()

content = [x.strip() for x in content]
content = [x.replace(" ", "") for x in content]
content = [x.split('\t') for x in content]
content = [[float(j) for j in i] for i in content]

In [4]:
# Klasa wierzcholkow
class Vertex:
    def __init__(self, name, source=False, sink=False):
        self.name = name
        self.source = source
        self.sink = sink

In [5]:
# Klasa krawedzi
class Edge:
    def __init__(self, start, end, capacity):
        '''Initialization of the edge'''
        self.start = start
        self.end = end
        self.capacity = capacity
        self.flow = 0
        self.returnEdge = None

    def getEnd():
        '''Return edge of graph'''
        return self.end

    def __repr__(self):
        return str(self.start) + " -> " + str(self.end) 
        # + " c:" + str(self.capacity)

## Zadanie 1
Zaimplementować  algorytm F-F zapisany za pomocą poniższego pseudokodu.

In [6]:
# Klasa odpowiadająca za przepływy
class FlowNetwork:
    def __init__(self):
        '''Initialize the Flow Network'''
        self.vertices = []
        self.network = {}
        self.adj = {}

    def getSource(self):
        '''Gets the source'''
        for vertex in self.vertices:
            if vertex.source == True:
                return vertex
        return None

    def getSink(self):
        '''Gets the sink'''
        for vertex in self.vertices:
            if vertex.sink == True:
                return vertex
        return None

    def setSource(self, name):
        for vertex in self.vertices:
            if name == vertex.name:
                vertex.source = True

    def setSink(self, name):
        for vertex in self.vertices:
            if name == vertex.name:
                vertex.sink = True;

    def getVertex(self, name):
        for vertex in self.vertices:
            if name == vertex.name:
                return vertex

    def vertexInNetwork(self, name):
        '''Check if the vertex already is in the network'''
        for vertex in self.vertices:
            if vertex.name == name:
                return True
        return False

    def addVertex(self, name, source=False, sink=False):
        '''Adds vertex to the flow, default sourse and sink are False'''
        if self.vertexInNetwork(name):
            return 0
        newVertex = Vertex(name, source, sink)
        self.vertices.append(newVertex)
        self.network[newVertex.name] = []
        self.adj[newVertex.name] = []

    def addEdge(self, start, end, capacity):
        '''Adds and edge with start, and and the capacity as parameters'''
        newEdge = Edge(start, end, capacity)
        returnEdge = Edge(end, start, 0)
        newEdge.returnEdge = returnEdge
        returnEdge.returnEdge = newEdge
        vertex = self.getVertex(start)
        self.network[vertex.name].append(newEdge)
        returnVertex = self.getVertex(end)
        self.network[returnVertex.name].append(returnEdge)
        self.adj[start].append(end)

    def bfs(self, start, end):
        '''Breadth First Search algo for paths, including visited instance so that algo will stuck in not loop'''
        queue = []
        visited = []
        visited.append(start)
        queue.append([(Edge(start, start, 0), 0)])
        while queue:
            path = queue.pop(0)
            node = path[-1][0].end
            if node == end:
                path.pop(0)
                return path

            for edge in self.network[node]:
                residualCapacity = edge.capacity - edge.flow
                #print(not (edge, residualCapacity) in path)
                
                if residualCapacity > 0 and not edge.end in visited:
                    new_path = list(path)
                    new_path.append((edge, residualCapacity))
                    #print(node)
                    queue.append(new_path)
                    visited.append(edge.end)
        return None

    def calculateMaxFlow(self):
        '''Calculate the max flow'''
        source = self.getSource()
        sink = self.getSink()
        #print(source.name)
        #print(sink.name)

        path = self.bfs(source.name, sink.name)
        while path != None:
            flow = min(edge[1] for edge in path)
            for edge, res in path:
                edge.flow += flow
                edge.returnEdge.flow -= flow
            path = self.bfs(source.name, sink.name)
            #print(path)
        return sum(edge.flow for edge in self.network[source.name])

In [7]:
def populate():
    '''Set the source to 43 and create network'''
    p = FlowNetwork()
    for i in content:
        p.addVertex(i[0])
        p.addVertex(i[1])
        p.addEdge(i[0],i[1],i[2])
    p.setSource(43)
    return p

## Zadanie 2
Obliczyć maksymalny przepływ dla grafu testowego, między wierzchołkami 43 i  180

In [8]:
_flow = populate()
_flow.setSink(180)
print("Max flow between 43 and 180: ", _flow.calculateMaxFlow())

Max flow between 43 and 180:  92.11999999999999


## Zadanie 3
Dla zadanego grafu testowego znaleźć wierzchołek, który będąc ujściem daje maksymalny przepływ dla sieci o źródle 43

In [11]:
# Przeliczam wszystkie przeplywy kontrolnie
result = []
przeplyw = populate()
for v in przeplyw.vertices:
    if v.name != 43:
        przeplyw = populate()
        przeplyw.setSink(v.name)
        max_flow = przeplyw.calculateMaxFlow()
        print("43 -",v.name, "maximum flow: ", max_flow)
        result.append((v.name, max_flow))

43 - 2.0 maximum flow:  0
43 - 106.0 maximum flow:  28.740000000000002
43 - 33.0 maximum flow:  0
43 - 143.0 maximum flow:  71.27
43 - 61.0 maximum flow:  16.87
43 - 161.0 maximum flow:  89.35999999999999
43 - 40.0 maximum flow:  0
43 - 103.0 maximum flow:  42.64
43 - 102.0 maximum flow:  19.17
43 - 45.0 maximum flow:  5.58
43 - 158.0 maximum flow:  88.06
43 - 75.0 maximum flow:  17.090000000000003
43 - 42.0 maximum flow:  0
43 - 84.0 maximum flow:  13.14
43 - 140.0 maximum flow:  74.64
43 - 69.0 maximum flow:  11.47
43 - 107.0 maximum flow:  37.97
43 - 138.0 maximum flow:  74.44
43 - 41.0 maximum flow:  0
43 - 112.0 maximum flow:  42.64
43 - 199.0 maximum flow:  93.69000000000001
43 - 176.0 maximum flow:  86.92999999999999
43 - 119.0 maximum flow:  48.31999999999999
43 - 144.0 maximum flow:  82.86999999999999
43 - 148.0 maximum flow:  78.66999999999999
43 - 194.0 maximum flow:  78.21000000000001
43 - 3.0 maximum flow:  0
43 - 160.0 maximum flow:  83.55
43 - 99.0 maximum flow:  19.32
4

In [10]:
a = max(result,key=lambda x:x[1])
print("Max flow from 43 is from " + str(a[0]) + "\n\rThe maximum flow is: " + str(a[1]))

Max flow from 43 is from 184.0
The maximum flow is: 100.10000000000001
